In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math

#20191127 アグリサポート大田原のデータでトウモロコシ・WCSのシミュレーション
#20191203 230haまで計算を追加
#最初に外生変数を定義→後の数式に直接値を入れても良いが、条件を変えてシミュレートする時に大変
#対象作物
area_corn=105*100 #トウモロコシ収穫面積　収穫時期は9月~10月
area_riceWCS=105*100 #WCS用稲　収穫時期は9月~10月
area_compost_area=0*100 #冬季散布

#作業適期間
operatable_term=61
#作業可能日確率
sunny_rate=0.82
#1日あたりの作業時間
day_working_time=8

#機械年焼却費
cost_of_machine_tractor_125ps=2350*10000/7*0.7
cost_of_machine_tractor_155ps=2800*10000/7
cost_of_machine_tractor_195ps=3200*10000/7*0.7
cost_of_machine_chemper_head3=850*10000/7
cost_of_machine_chemper_head4=1150*10000/7
cost_of_machine_wheel_loader=600*10000/7 #グラブ装着
cost_of_machine_wheel_loader2=550*10000/7 #バケット装着
cost_of_machine_compactor=2800*10000/7
cost_of_machine_tarctor_130ps=1200*10000/7*0.7
cost_of_machine_self_propelled_harvester=1900*10000/7
cost_of_machine_self_propelled_wrapper=350*10000/7

#ダンプ借料（1aあたり）
dump_rental=539/10

#資材費・燃料費（1aあたり）
yanmer_system_variable=4242/10
takakita_system_variable=8837/10

#人件費（）
cost_of_daylabor=1500 #臨時雇用の時間あたり労賃
cost_of_fulltimelabor=1000000 #通年雇用の年間労賃


#作業パラメータ（１時間当たりの作業可能面積(a)）
operation_ef_unit_corn_harvest_yanmar1=225/8 #ユニット6人一組,日作業可能時間8h，作業効率, 根拠：2019/11/25メール、以下同
operation_ef_unit_riceWCS_harvest_yanmar1=227/8 #ユニット6人一組,日作業可能時間8h，作業効率
operation_ef_unit_corn_harvest_yanmar2=193/8 #ユニット６人組，日作業可能時間8h，作業効率
operation_ef_unit_riceWCS_harvest_yanmar2=193/8 #ユニット６人組，日作業可能時間8h，作業効率
operation_ef_unit_corn_harvest_takakita=113/8  #ユニット６人組，日作業可能時間8h，作業効率
operation_ef_unit_riceWCS_harvest_takakita=116/8 #ユニット６人組，日作業可能時間8h，作業効率
#operation_ef_unit_riceWCS_harvest_takakita=18.6*100/82.4

model = ConcreteModel()


#機械関係の変数（台数）
model.number_of_machine_tractor_125ps=Var(initialize=1,within=NonNegativeIntegers) #１２5psトラクターの台数
#model.number_of_machine_tractor_155ps=Var(initialize=1,within=NonNegativeIntegers) #１55psトラクターの台数
model.number_of_machine_tractor_195ps=Var(initialize=1,within=NonNegativeIntegers) #１95psトラクターの台数：ヤンマーシステム
model.number_of_machine_chemper_head3=Var(initialize=0,within=NonNegativeIntegers) #ケンパーヘッド３の台数
model.number_of_machine_chemper_head4=Var(initialize=1,within=NonNegativeIntegers) #ケンパーヘッド4の台数：ヤンマーシステム
model.number_of_machine_wheel_loader1=Var(initialize=2,within=NonNegativeIntegers) #ホイルローダー1の台数：ヤンマーシステム、タカキタシステム（２台）
model.number_of_machine_wheel_loader2=Var(initialize=1,within=NonNegativeIntegers) #ホイルローダー2の台数：ヤンマーシステム
model.number_of_machine_compactor=Var(initialize=1,within=NonNegativeIntegers) #マルチコンパクターの台数：ヤンマーシステム
model.number_of_machine_tractor_130ps=Var(initialize=1,within=NonNegativeIntegers) #１30psトラクターの台数：ヤンマーシステム
model.number_of_machine_self_propelled_harvester=Var(initialize=2,within=NonNegativeIntegers) #汎用収穫機台数：タカキタシステム
model.number_of_machine_self_propelled_wrapper=Var(initialize=2,within=NonNegativeIntegers) #自走式ラッピングマシン台数：タカキタシステム

#通年雇用人数→年間通した作業を行わない限りは、０人になる。時給で必要時間だけ雇う方が安いから。ただし、事故対応を考えるとリスクはある。
model.number_of_fulltimelabor=Var(initialize=5,within=NonNegativeIntegers)
model.number_of_fulltimelabor_yanmar1=Var(initialize=5,within=NonNegativeIntegers)
model.number_of_fulltimelabor_yanmar2=Var(initialize=5,within=NonNegativeIntegers)
model.number_of_fulltimelabor_takakita=Var(initialize=5,within=NonNegativeIntegers)

#臨時雇用人数
model.number_of_daylabor_yanmar1=Var(initialize=0,within=NonNegativeIntegers)
model.number_of_daylabor_yanmar2=Var(initialize=0,within=NonNegativeIntegers)
model.number_of_daylabor_takakita=Var(initialize=0,within=NonNegativeIntegers)

#ヤンマーシステムユニット数（作業班の数。この数に応じて必要な要員・機械を調達する）
model.number_of_unit_yanmar1=Var(initialize=1,within=NonNegativeIntegers)
model.number_of_unit_yanmar2=Var(initialize=1,within=NonNegativeIntegers)

#タカキタシステムユニット数（作業班の数。この数に応じて必要な要員・機械を調達する）
model.number_of_unit_takakita=Var(initialize=1,within=NonNegativeIntegers)

#ヤンマーシステムのユニット当たり作業時間
model.hour_per_unit_corn_harvest_yanmar1=Var(initialize=1,within=NonNegativeReals)
model.hour_per_unit_corn_harvest_yanmar2=Var(initialize=1,within=NonNegativeReals)
model.hour_per_unit_riceWCS_harvest_yanmar1=Var(initialize=1,within=NonNegativeReals)
model.hour_per_unit_riceWCS_harvest_yanmar2=Var(initialize=1,within=NonNegativeReals)
#タカキタシステムのユニット当たり作業時間
model.hour_per_unit_corn_harvest_takakita=Var(initialize=1,within=NonNegativeReals)
model.hour_per_unit_riceWCS_harvest_takakita=Var(initialize=1,within=NonNegativeReals)
#ヤンマーシステムのユニット当たり作業面積
model.area_per_unit_corn_harvest_yanmar1=Var(initialize=1,within=NonNegativeReals)
model.area_per_unit_corn_harvest_yanmar2=Var(initialize=1,within=NonNegativeReals)
model.area_per_unit_riceWCS_harvest_yanmar1=Var(initialize=1,within=NonNegativeReals)
model.area_per_unit_riceWCS_harvest_yanmar2=Var(initialize=1,within=NonNegativeReals)
#タカキタシステムのユニット当たり作業面積
model.area_per_unit_corn_harvest_takakita=Var(initialize=1,within=NonNegativeReals)
model.area_per_unit_riceWCS_harvest_takakita=Var(initialize=1,within=NonNegativeReals)


#目標関数（資材費＋燃料費、労働費、機械償却費を最小化。作業面積は後の制約式で与える）
model.obj=Objective(expr=model.number_of_unit_yanmar1*model.area_per_unit_corn_harvest_yanmar1*yanmer_system_variable
                    +model.number_of_unit_yanmar2*model.area_per_unit_corn_harvest_yanmar2*yanmer_system_variable
                    +model.number_of_unit_yanmar1*model.area_per_unit_riceWCS_harvest_yanmar1*yanmer_system_variable
                    +model.number_of_unit_yanmar2*model.area_per_unit_riceWCS_harvest_yanmar2*yanmer_system_variable
                    +model.number_of_unit_takakita*model.area_per_unit_corn_harvest_takakita*takakita_system_variable
                    +model.number_of_unit_takakita*model.area_per_unit_riceWCS_harvest_takakita*takakita_system_variable
                    +model.number_of_machine_tractor_125ps*cost_of_machine_tractor_125ps
                    +model.number_of_machine_tractor_195ps*cost_of_machine_tractor_195ps
                    +model.number_of_machine_chemper_head3*cost_of_machine_chemper_head3
                    +model.number_of_machine_chemper_head4*cost_of_machine_chemper_head4
                    +model.number_of_machine_wheel_loader1*cost_of_machine_wheel_loader
                    +model.number_of_machine_wheel_loader2*cost_of_machine_wheel_loader2
                    +model.number_of_machine_compactor*cost_of_machine_compactor
                    +model.number_of_machine_tractor_130ps*cost_of_machine_tarctor_130ps
                    +model.number_of_machine_self_propelled_harvester*cost_of_machine_self_propelled_harvester
                    +model.number_of_machine_self_propelled_wrapper*cost_of_machine_self_propelled_wrapper
                    +(area_corn+area_riceWCS)*dump_rental
                    +model.hour_per_unit_corn_harvest_yanmar1*model.number_of_daylabor_yanmar1*cost_of_daylabor
                    +model.hour_per_unit_corn_harvest_yanmar2*model.number_of_daylabor_yanmar2*cost_of_daylabor
                    +model.hour_per_unit_riceWCS_harvest_yanmar1*model.number_of_daylabor_yanmar1*cost_of_daylabor
                    +model.hour_per_unit_riceWCS_harvest_yanmar2*model.number_of_daylabor_yanmar2*cost_of_daylabor
                    +model.hour_per_unit_corn_harvest_takakita*model.number_of_daylabor_takakita*cost_of_daylabor
                    +model.hour_per_unit_riceWCS_harvest_takakita*model.number_of_daylabor_takakita*cost_of_daylabor
                    +model.number_of_fulltimelabor*cost_of_fulltimelabor
                    ,sense= minimize)

#受託面積(それぞれの作業ユニットの作業面積合計が全体面積に等しくなるように)
model.const_area_corn_yanmar1=Constraint(expr=model.area_per_unit_corn_harvest_yanmar1==model.hour_per_unit_corn_harvest_yanmar1
                                         *operation_ef_unit_corn_harvest_yanmar1)
model.const_area_corn_yanmar2=Constraint(expr=model.area_per_unit_corn_harvest_yanmar2==model.hour_per_unit_corn_harvest_yanmar2
                                         *operation_ef_unit_corn_harvest_yanmar2)
model.const_area_corn_takakita=Constraint(expr=model.area_per_unit_corn_harvest_takakita==model.hour_per_unit_corn_harvest_takakita
                                         *operation_ef_unit_corn_harvest_takakita)
model.const_area_corn_total=Constraint(expr=area_corn==model.number_of_unit_yanmar1*model.area_per_unit_corn_harvest_yanmar1
                                      +model.number_of_unit_yanmar2*model.area_per_unit_corn_harvest_yanmar2
                                      +model.number_of_unit_takakita*model.area_per_unit_corn_harvest_takakita)

model.const_area_riceWCS_yanmar1=Constraint(expr=model.area_per_unit_riceWCS_harvest_yanmar1==model.hour_per_unit_riceWCS_harvest_yanmar1
                                         *operation_ef_unit_riceWCS_harvest_yanmar1)
model.const_area_riceWCS_yanmar2=Constraint(expr=model.area_per_unit_riceWCS_harvest_yanmar2==model.hour_per_unit_riceWCS_harvest_yanmar2
                                         *operation_ef_unit_riceWCS_harvest_yanmar2)
model.const_area_riceWCS_takakita=Constraint(expr=model.area_per_unit_riceWCS_harvest_takakita==model.hour_per_unit_riceWCS_harvest_takakita
                                         *operation_ef_unit_riceWCS_harvest_takakita)
model.const_area_riceWCS_total=Constraint(expr=area_riceWCS==model.number_of_unit_yanmar1*model.area_per_unit_riceWCS_harvest_yanmar1
                                      +model.number_of_unit_yanmar2*model.area_per_unit_riceWCS_harvest_yanmar2
                                      +model.number_of_unit_takakita*model.area_per_unit_riceWCS_harvest_takakita)


#作業可能性制約（ユニットの作業可能な時間は、作業適期日数×日作業可能時間×晴れ確率以下、という制約）
model.const_operatable_yanmar1=Constraint(expr=model.hour_per_unit_corn_harvest_yanmar1+model.hour_per_unit_riceWCS_harvest_yanmar1
                                          <=operatable_term*day_working_time*sunny_rate)
model.const_operatable_yanmar2=Constraint(expr=model.hour_per_unit_corn_harvest_yanmar2+model.hour_per_unit_riceWCS_harvest_yanmar2
                                          <=operatable_term*day_working_time*sunny_rate)
model.const_operatable_takakita=Constraint(expr=model.hour_per_unit_corn_harvest_takakita+model.hour_per_unit_riceWCS_harvest_takakita
                                           <=operatable_term*day_working_time*sunny_rate)

#要員数需給
model.const_labor_total=Constraint(expr=model.number_of_unit_yanmar1*6
                                   +model.number_of_unit_yanmar2*6
                                   +model.number_of_unit_takakita*6
                                   <=model.number_of_daylabor_yanmar1
                                   +model.number_of_daylabor_yanmar2
                                   +model.number_of_daylabor_takakita
                                   +model.number_of_fulltimelabor_yanmar1
                                   +model.number_of_fulltimelabor_yanmar2
                                   +model.number_of_fulltimelabor_takakita)

model.const_labor_yanmar1=Constraint(expr=model.number_of_unit_yanmar1*6<=model.number_of_daylabor_yanmar1
                                          +model.number_of_fulltimelabor_yanmar1)
model.const_labor_yanmar2=Constraint(expr=model.number_of_unit_yanmar2*6<=model.number_of_daylabor_yanmar2
                                         +model.number_of_fulltimelabor_yanmar2)
model.const_labor_takakita=Constraint(expr=model.number_of_unit_takakita*6<=model.number_of_daylabor_takakita
                                          +model.number_of_fulltimelabor_takakita)

model.const_fulltimelabor=Constraint(expr=model.number_of_fulltimelabor==model.number_of_fulltimelabor_yanmar1+model.number_of_fulltimelabor_yanmar2+model.number_of_fulltimelabor_takakita)

#労働時間需給
model.const_labor_hour=Constraint(expr=model.number_of_unit_yanmar1*model.hour_per_unit_corn_harvest_yanmar1*6
                                  +model.number_of_unit_yanmar2*model.hour_per_unit_corn_harvest_yanmar2*6
                                  +model.number_of_unit_takakita*model.hour_per_unit_corn_harvest_takakita*6
                                 +model.number_of_unit_yanmar1*model.hour_per_unit_riceWCS_harvest_yanmar1*6
                                  +model.number_of_unit_yanmar2*model.hour_per_unit_riceWCS_harvest_yanmar2*6
                                  +model.number_of_unit_takakita*model.hour_per_unit_riceWCS_harvest_takakita*6
                                 <=model.number_of_daylabor_yanmar1*operatable_term*day_working_time*sunny_rate
                                 +model.number_of_daylabor_yanmar2*operatable_term*day_working_time*sunny_rate
                                 +model.number_of_daylabor_takakita*operatable_term*day_working_time*sunny_rate
                                 +model.number_of_fulltimelabor_yanmar1*operatable_term*day_working_time*sunny_rate
                                   +model.number_of_fulltimelabor_yanmar2*operatable_term*day_working_time*sunny_rate
                                   +model.number_of_fulltimelabor_takakita*operatable_term*day_working_time*sunny_rate)


#機械台数需給→ユニット数と機械台数を関連づけないとユニット数・ユニットあたり稼働時間が一意に決まらない。延稼働時間と台数の識別性の問題も
model.const_tractor_195ps_unit=Constraint(expr=model.number_of_unit_yanmar1<=model.number_of_machine_tractor_195ps)
model.const_chemper4_unit=Constraint(expr=model.number_of_unit_yanmar1<=model.number_of_machine_chemper_head4) 
model.const_tractor_125ps_unit=Constraint(expr=model.number_of_unit_yanmar2<=model.number_of_machine_tractor_125ps)
model.const_chemper3_unit=Constraint(expr=model.number_of_unit_yanmar2<=model.number_of_machine_chemper_head3) 
model.const_compactor_unit=Constraint(expr=model.number_of_unit_yanmar1
                                           +model.number_of_unit_yanmar2<=model.number_of_machine_compactor)
model.const_taractor_130ps_unit=Constraint(expr=model.number_of_unit_yanmar1
                                           +model.number_of_unit_yanmar2<=model.number_of_machine_tractor_130ps)
model.const_propelled_harvester_unit=Constraint(expr=model.number_of_unit_takakita
                                                     <=model.number_of_machine_self_propelled_harvester)
model.const_propelled_wrapper_unit=Constraint(expr=model.number_of_unit_takakita
                                                   <=model.number_of_machine_self_propelled_wrapper)
model.const_wheel_loader1_unit_corn=Constraint(expr=model.number_of_unit_yanmar1
                                               +model.number_of_unit_yanmar2
                                               +model.number_of_unit_takakita*2<=model.number_of_machine_wheel_loader1)
model.const_wheel_loader2_unit=Constraint(expr=model.number_of_unit_yanmar1
                                               +model.number_of_unit_yanmar2<=model.number_of_machine_wheel_loader2)

#機械稼働時間受給→無いと解が収束しない
model.const_hour_tractor_195ps=Constraint(expr=model.number_of_unit_yanmar1
                                          *model.hour_per_unit_corn_harvest_yanmar1
                                          +model.number_of_unit_yanmar1
                                          *model.hour_per_unit_riceWCS_harvest_yanmar1
                                          <=model.number_of_machine_tractor_195ps*operatable_term*day_working_time*sunny_rate)
model.const_hour_chemper_head4=Constraint(expr=model.number_of_unit_yanmar1
                                          *model.hour_per_unit_corn_harvest_yanmar1
                                          +model.number_of_unit_yanmar1
                                          *model.hour_per_unit_riceWCS_harvest_yanmar1
                                          <=model.number_of_machine_chemper_head4*operatable_term*day_working_time*sunny_rate)
model.const_hour_tractor_125ps=Constraint(expr=model.number_of_unit_yanmar2
                                          *model.hour_per_unit_corn_harvest_yanmar2
                                          +model.number_of_unit_yanmar2
                                          *model.hour_per_unit_riceWCS_harvest_yanmar2
                                          <=model.number_of_machine_tractor_125ps*operatable_term*day_working_time*sunny_rate)
model.const_hour_chemper_head3=Constraint(expr=model.number_of_unit_yanmar2
                                          *model.hour_per_unit_corn_harvest_yanmar2
                                          +model.number_of_unit_yanmar2
                                          *model.hour_per_unit_riceWCS_harvest_yanmar2
                                          <=model.number_of_machine_chemper_head3*operatable_term*day_working_time*sunny_rate)                                       
                                          
model.const_hour_wheel_loader1=Constraint(expr=model.number_of_unit_yanmar1*model.hour_per_unit_corn_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_corn_harvest_yanmar2
                                    +model.number_of_unit_yanmar1*model.hour_per_unit_riceWCS_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_riceWCS_harvest_yanmar2
                                    +model.number_of_unit_takakita*model.hour_per_unit_corn_harvest_takakita*2
                                    +model.number_of_unit_takakita*model.hour_per_unit_riceWCS_harvest_takakita*2
                                    <=model.number_of_machine_wheel_loader1*operatable_term*day_working_time*sunny_rate)
model.const_hour_wheel_loader2=Constraint(expr=model.number_of_unit_yanmar1*model.hour_per_unit_corn_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_corn_harvest_yanmar2
                                    +model.number_of_unit_yanmar1*model.hour_per_unit_riceWCS_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_riceWCS_harvest_yanmar2
                                    <=model.number_of_machine_wheel_loader2*operatable_term*day_working_time*sunny_rate)
model.const_hour_compactor=Constraint(expr=model.number_of_unit_yanmar1*model.hour_per_unit_corn_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_corn_harvest_yanmar2
                                    +model.number_of_unit_yanmar1*model.hour_per_unit_riceWCS_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_riceWCS_harvest_yanmar2
                                    <=model.number_of_machine_compactor*operatable_term*day_working_time*sunny_rate)
model.const_hour_taractor_130ps=Constraint(expr=model.number_of_unit_yanmar1*model.hour_per_unit_corn_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_corn_harvest_yanmar2
                                    +model.number_of_unit_yanmar1*model.hour_per_unit_riceWCS_harvest_yanmar1
                                    +model.number_of_unit_yanmar2*model.hour_per_unit_riceWCS_harvest_yanmar2
                                    <=model.number_of_machine_tractor_130ps*operatable_term*day_working_time*sunny_rate)
model.const_hour_propelled_harvester=Constraint(expr=+model.number_of_unit_takakita*model.hour_per_unit_corn_harvest_takakita
                                    +model.number_of_unit_takakita*model.hour_per_unit_riceWCS_harvest_takakita
                                          <=model.number_of_machine_self_propelled_harvester*operatable_term*day_working_time*sunny_rate)
model.const_hour_propelled_wrapper=Constraint(expr=+model.number_of_unit_takakita*model.hour_per_unit_corn_harvest_takakita
                                    +model.number_of_unit_takakita*model.hour_per_unit_riceWCS_harvest_takakita
                                          <=model.number_of_machine_self_propelled_wrapper*operatable_term*day_working_time*sunny_rate)

#計算時間短縮のための制約
model.const_unit_yanmer1=Constraint(expr=model.number_of_unit_yanmar1<=5)
model.const_unit_yanmer2=Constraint(expr=model.number_of_unit_yanmar2<=5)
model.const_unit_takakita=Constraint(expr=model.number_of_unit_takakita<=5)
# model.const_unit_yanmer1_moving_corn=Constraint(expr=model.hour_per_unit_corn_harvest_yanmar1>=1)
# model.const_unit_yanmer2_moving_corn=Constraint(expr=model.hour_per_unit_corn_harvest_yanmar2>=1)
# model.const_unit_takakita_moving_corn=Constraint(expr=model.hour_per_unit_corn_harvest_takakita>=1)
#model.const_unit_yanmer1_moving_riceWCS=Constraint(expr=model.hour_per_unit_riceWCS_harvest_yanmar1>=1)
#model.const_unit_yanmer2_moving_riceWCS=Constraint(expr=model.hour_per_unit_riceWCS_harvest_yanmar2>=1)
#model.const_unit_takakita_moving_riceWCS=Constraint(expr=model.hour_per_unit_riceWCS_harvest_takakita>=1)
#ここの設定をしないと、臨時雇用しか選ばれない
# model.const_fulltime_labor=Constraint(expr=model.number_of_fulltimelabor==2)

#model.const_takakita_choice=Constraint(expr=model.number_of_unit_takakita>=1)


opt = SolverFactory("scipampl")
result = opt.solve(model, tee=False)
model.display()











Model unknown

  Variables:
    number_of_machine_tractor_125ps : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeIntegers
    number_of_machine_tractor_195ps : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeIntegers
    number_of_machine_chemper_head3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeIntegers
    number_of_machine_chemper_head4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeIntegers
    number_of_machine_wheel_loader1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :  None : False : False : NonNegativeIntegers
  